# Bachelor's thesis

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import parquet
from sklearn.preprocessing import StandardScaler
# statistical tests
from scipy.stats import mannwhitneyu
from scipy.stats import ks_2samp

## Loading the data 

In [2]:
accepts = pd.DataFrame()
accepts = df = pd.read_parquet("../data/accepts_original.parquet", engine="fastparquet")

In [3]:
accepts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 2.5+ GB


In [4]:
rejects = pd.DataFrame()
rejects = pd.read_parquet('../data/rejects_original.parquet', engine="fastparquet")

## Preprocessing

### First we preprocess the rejects. We delete FICO values, who can not occur and delete heavy outliers in the variable dti.

In [5]:
rejects = rejects.drop(["Application Date","Loan Title","Zip Code","Policy Code"], axis = 1)

In [6]:
rejects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27648741 entries, 0 to 27648740
Data columns (total 5 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Amount Requested      float64
 1   Risk_Score            float64
 2   Debt-To-Income Ratio  object 
 3   State                 object 
 4   Employment Length     object 
dtypes: float64(2), object(3)
memory usage: 1.0+ GB


In [7]:
rejects.rename(columns={'Amount Requested': 'loan_amnt', 'Risk_Score': 'fico', 'Debt-To-Income Ratio': 'dti', 'State': 'addr_state', 'Employment Length': 'emp_length'}, inplace=True)

In [8]:
rejects.dropna(subset=['fico', 'emp_length'], inplace=True)

In [9]:
rejects['dti'] = rejects['dti'].str.replace('%', '').astype(float)
rejects = rejects[
    (rejects['dti'] >= 0) & 
    (rejects['fico'] <= 850) & 
    (rejects['fico'] >= 300) & 
    (np.abs(stats.zscore(rejects["dti"])) < 2)
]

In [10]:
def categorize_emp_length(x):
    if(x=='< 1 year'):
        return 0
    if(x=='1 year'):
        return 1
    if(x=='2 years'):
        return 2
    if(x=='3 years'):
        return 3
    if(x=='4 years'):
        return 4
    if(x=='5 years'):
        return 5
    if(x=='6 years'):
        return 6
    if(x=='7 years'):
        return 7
    if(x=='8 years'):
        return 8
    if(x=='9 years'):
        return 9
    return 10

In [11]:
rejects['emp_length'] = rejects['emp_length'].apply(categorize_emp_length)
rejects.describe().apply(lambda x: x.apply('{0:.1f}'.format))

,loan_amnt,fico,dti,emp_length
count,8863111.0,8863111.0,8863111.0,8863111.0
mean,12664.8,632.7,71.2,0.5
std,10897.1,64.3,748.7,1.9
min,0.0,300.0,0.0,0.0
25%,4000.0,592.0,9.4,0.0
50%,10000.0,637.0,20.8,0.0
75%,20000.0,674.0,36.4,0.0
max,1400000.0,850.0,35700.0,10.0


## Accepts

In [12]:
accepts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 2.5+ GB


### First I drop all features I dont need and drop all missing values in loan_status since this is the most valuable variable. Then I drop the missing values in my other variables. For dti I drop heavy outliers.

In [13]:
rejects.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8863111 entries, 0 to 27648740
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   loan_amnt   float64
 1   fico        float64
 2   dti         float64
 3   addr_state  object 
 4   emp_length  int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 405.7+ MB


In [14]:
accepts = accepts[["loan_amnt","fico_range_low","dti","addr_state","emp_length","loan_status", "int_rate","acc_open_past_24mths","bc_open_to_buy"]]

In [15]:
accepts.rename(columns={'fico_range_low': 'fico'}, inplace=True)

/var/folders/gq/_cdvcqp945l0bvvrdmswyvs80000gn/T/ipykernel_13117/885549311.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accepts.rename(columns={'fico_range_low': 'fico'}, inplace=True)


In [16]:
accepts[["loan_amnt","fico","dti","addr_state","emp_length"]].describe().apply(lambda x: x.apply('{0:.1f}'.format))

,loan_amnt,fico,dti
count,2260668.0,2260668.0,2258957.0
mean,15046.9,698.6,18.8
std,9190.2,33.0,14.2
min,500.0,610.0,-1.0
25%,8000.0,675.0,11.9
50%,12900.0,690.0,17.8
75%,20000.0,715.0,24.5
max,40000.0,845.0,999.0


### Now I categorize my variables.

In [17]:
accepts = accepts[~accepts.loan_status.isin([
    "Does not meet the credit policy. Status:Fully Paid",
    "Does not meet the credit policy. Status:Charged Off"
])]

def categorize_loan(x):
    if(x=='Fully Paid' or x=='Late (31-120 days)' or x=='In Grace Period' or x=="Late (16-30 days)"):
        return 0
    if (x=="Current"):
        return 2
    return 1
accepts['loan_status'] = accepts['loan_status'].apply(categorize_loan)

In [18]:
accepts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2257952 entries, 0 to 2260700
Data columns (total 9 columns):
 #   Column                Dtype  
---  ------                -----  
 0   loan_amnt             float64
 1   fico                  float64
 2   dti                   float64
 3   addr_state            object 
 4   emp_length            object 
 5   loan_status           int64  
 6   int_rate              float64
 7   acc_open_past_24mths  float64
 8   bc_open_to_buy        float64
dtypes: float64(6), int64(1), object(2)
memory usage: 172.3+ MB


In [19]:
accepts.dropna(subset=['loan_status', 'dti', "emp_length"], inplace=True)

In [20]:
accepts = accepts[
    (accepts['dti'] >= 0) & 
    (np.abs(stats.zscore(accepts["dti"])) < 3)
]

In [21]:
accepts['emp_length'] = accepts['emp_length'].apply(categorize_emp_length)

### Here I also drop the missing values for my unobserved variables and drop the current loans because we do not know if they will default or not.

In [22]:
accepts.isnull().sum()/accepts.shape[0]*100

loan_amnt               0.000000
fico                    0.000000
dti                     0.000000
addr_state              0.000000
emp_length              0.000000
loan_status             0.000000
int_rate                0.000000
acc_open_past_24mths    2.184582
bc_open_to_buy          3.236954
dtype: float64

In [23]:
accepts = accepts[accepts.loan_status != 2]

In [24]:
accepts[["loan_amnt","fico","dti","addr_state","emp_length"]].describe().apply(lambda x: x.apply('{0:.1f}'.format))

,loan_amnt,fico,dti,emp_length
count,1296365.0,1296365.0,1296365.0,1296365.0
mean,14662.8,696.0,18.1,6.0
std,8777.7,31.6,8.5,3.7
min,500.0,625.0,0.0,0.0
25%,8000.0,670.0,11.8,2.0
50%,12325.0,690.0,17.5,6.0
75%,20000.0,710.0,23.9,10.0
max,40000.0,845.0,54.0,10.0


In [25]:
accepts.dropna(subset=['bc_open_to_buy','acc_open_past_24mths', 'int_rate'], inplace=True)

### Now I bin the variable address state according to its prior default rate into 4 subgroups. Meaning a state with a low default rate goes into bin 0 and a state with a high default rate goes into bin 3.

In [26]:
default_rate = accepts.groupby('addr_state')['loan_status'].mean()
bins = pd.qcut(default_rate, q=4, labels=False)
binned_states = default_rate.to_frame().assign(Bin=bins).reset_index()
accepts = accepts.merge(binned_states[['addr_state', 'Bin']], on='addr_state', how='left')

In [27]:
accepts = accepts.drop("addr_state", axis=1)
accepts.rename(columns={'Bin': 'addr_state'}, inplace=True)

In [28]:
accepts = accepts[["loan_amnt","fico","dti","addr_state","emp_length","loan_status", "int_rate","acc_open_past_24mths","bc_open_to_buy"]]

### These are my final accepts, which I use for the data generation process.

In [29]:
accepts.to_csv('../data/accepts.csv', encoding='utf-8', index=False)

## Statistical Tests

### Now I do some statistical tests on the original accepts and rejects. Here you can see the Mann-Whitney-U-Test for the variable loan amount as an example. I used the statistic to calculate my effect sizes.

In [30]:
accepts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1237419 entries, 0 to 1237418
Data columns (total 9 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   loan_amnt             1237419 non-null  float64
 1   fico                  1237419 non-null  float64
 2   dti                   1237419 non-null  float64
 3   addr_state            1237419 non-null  int64  
 4   emp_length            1237419 non-null  int64  
 5   loan_status           1237419 non-null  int64  
 6   int_rate              1237419 non-null  float64
 7   acc_open_past_24mths  1237419 non-null  float64
 8   bc_open_to_buy        1237419 non-null  float64
dtypes: float64(6), int64(3)
memory usage: 85.0 MB


In [31]:
rejects.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8863111 entries, 0 to 27648740
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   loan_amnt   float64
 1   fico        float64
 2   dti         float64
 3   addr_state  object 
 4   emp_length  int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 405.7+ MB


In [32]:
mannwhitneyu(accepts.iloc[:,0], rejects.iloc[:,0])

MannwhitneyuResult(statistic=6625489065186.0, pvalue=0.0)